In [1]:
import tensorflow as tf
from tensorflow.keras import models,layers 
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
img_size=256
batch_size=32


In [4]:
dataset=tf.keras.preprocessing.image_dataset_from_directory (
    "/content/drive/MyDrive/Colab Notebooks/Disease Detection/Training",
    shuffle=True,
    image_size=(img_size,img_size),
    batch_size= batch_size
)

Found 3251 files belonging to 3 classes.


In [5]:
class_names=dataset.class_names
class_names

['Early_Blight', 'Healthy', 'Late_Blight']

In [6]:
len(dataset)

102

In [7]:
def dataset_partitions(ds,train_split=0.8,val_split=0.1,shuffle=True,shuffle_size=10000):
  
  ds_size=len(ds)

  if shuffle:
    ds=ds.shuffle(shuffle_size,seed=12)

  train_size=int(train_split*ds_size)
  val_size=int(val_split * ds_size)

  train_ds=ds.take(train_size)

  val_ds=ds.skip(train_size).take(val_size)
  test_ds=ds.skip(train_size).skip(val_size)
  
  return train_ds,val_ds,test_ds

In [8]:
train_ds,val_ds,test_ds=dataset_partitions(dataset)

In [9]:
len(train_ds)

81

In [10]:
len(val_ds)

10

In [11]:
len(test_ds)

11

In [12]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [13]:
resize_and_rescale=tf.keras.Sequential([
                      layers.experimental.preprocessing.Resizing(img_size,img_size),
                      layers.experimental.preprocessing.Rescaling(1.0/255)

])

In [14]:
   model=models.Sequential([
                           resize_and_rescale,
                          layers.Conv2D(32,(3,3),activation='relu',input_shape=(batch_size,img_size,img_size,3)), 
                           layers.MaxPooling2D((2,2)),
                            layers.Conv2D(64,(3,3),activation='relu'), 
                           layers.MaxPooling2D((2,2)),
                            layers.Conv2D(64,(3,3),activation='relu'), 
                           layers.MaxPooling2D((2,2)),
                            layers.Conv2D(64,(3,3),activation='relu'), 
                           layers.MaxPooling2D((2,2)),
                            layers.Conv2D(64,(3,3),activation='relu'), 
                           layers.MaxPooling2D((2,2)),
   
                            layers.Flatten(),
                            layers.Dense(64,activation="relu"),
                            layers.Dense(3,activation='softmax')
   
   
   ])

   model.build(input_shape=(batch_size,img_size,img_size,3))

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 127, 127, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 62, 62, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (32, 60, 60, 64)         

In [16]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'] 
)

In [ ]:
history=model.fit( 
    train_ds, 
    epochs=10, 
    batch_size=batch_size,
     verbose=1,
      validation_data=val_ds 
)

Epoch 1/10


In [ ]:
scores=model.evaluate(test_ds)

In [ ]:
scores


In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

In [ ]:
import numpy as np
for images,labels in test_ds.take(1):
 
  
  firstimage=images[0].numpy().astype('uint8')
  firstlabel=labels[0].numpy()  

  plt.imshow(firstimage) 
  plt.axis('off')
  print("Actual:",class_names[firstlabel])
  prediction=model.predict(images)
  print("Predicted:",class_names[np.argmax(prediction[0])])